Task
- Add in other uestions
-- Refactor code for the aggregation of timely activity
- Fix the plots
- Start on EDA
- Model selection

Background Information on Dataset: BLAH BLAH BLAH

Importing Packages

In [184]:
import numpy as np
import pandas as pd
import dask.dataframe as dd
import matplotlib
import plotly
import plotly.express as px
import plotly.graph_objects as go

from datetime import datetime

EDA

Load in dataset

Aside: Let's measure how long it takes to load in our dataset: using pandas and dask

In [ ]:
import cProfile
import pstats

profiler = cProfile.Profile()
profiler.enable()
df = pd.read_parquet("Data/Large/articles.parquet")
profiler.disable()

profiler_stats = pstats.Stats(profiler)
profiler_stats.print_stats()

In [ ]:
profiler = cProfile.Profile()
profiler.enable()
df = dd.read_parquet("Data/Large/articles.parquet", engine = "fastparquet")
profiler.disable()

profiler_stats = pstats.Stats(profiler)
profiler_stats.print_stats()

Even though the dataframe took 1.464 seconds to load using pandas, dask took 0.054 seconds. This is a huge speedup ~ 27x

In [185]:
#Load in various dataframes
## Articles
df_art = pd.read_parquet("Data/Small/articles.parquet")

## Behaviors
df_bev = pd.read_parquet("Data/Small/train/behaviors.parquet")

## History
df_his = pd.read_parquet("Data/Small/train/history.parquet")



Join the data sources

In [186]:
# Convert datatype of column first
df_bev['article_ids_clicked'] = df_bev['article_ids_clicked'].apply(lambda x: x[0])

In [187]:
# Join bevhaiors to article
df= df_bev.join(df_art.set_index("article_id"), on = "article_ids_clicked")

# Join bevhaiors to history 
df= df.join(df_his.set_index("user_id"), on = "user_id")

# Drop all other dataframes from me
df_bev = []
df_his = []
df_art = []

In [211]:
# Preprocessing
df.dropna(subset=['article_id'], inplace= True)



df['article_id'] = df['article_id'].apply(lambda x: int(x))

df['article_id']= df['article_id'].astype(np.int64)



Visualizations

In [189]:
def plotly_index_values(groupby_object):
    """Takes in a clause from a pandas groupby statement and returns X and Y variables used for plotting"""

    index = groupby_object.index
    values = groupby_object.values

    return index, values



Lets calculate the unique users for hourly, daily, and day of the week. Let's use a subset of the data until we know our plots are very good

#### Biggest thing is user engagement : Bigger User Engagement -> More eveneue
#### We need to maximize the amount of ads these guys are viewing -> this leads on to them clicking on new articles for ads
#### So, let's not make article length too short so that people can maximize their session lengths with a lot of articles!

In [190]:
# Let's make subsets for each user_id and then populate the frequency based on that

## Get all unique ids in a list

unique_user_ids = df['user_id'].values[0:1000]
unique_user_ids = set(unique_user_ids)


# Create dictionaries
unique_users_daily_freq = {}
unique_users_hourly_freq = {}
unique_users_dayofweek_freq = {}
unique_users_weekly_freq = {}

for id in unique_user_ids:
    # Get the subset of that user id
    tmp_df = df[df['user_id'] == id]

    # Now lets go through each and populate the unique dates, hours and day of the week for each user
    dates = []
    hours = []
    dayofweek = []
    week = []
    indices = np.array(tmp_df.index)
    for i in indices:
        tmp_datetime = pd.DatetimeIndex(tmp_df['impression_time_fixed'][i])
        tmp_date = tmp_datetime.date
        tmp_time = tmp_datetime.time
        tmp_dayofweek = tmp_datetime.day_of_week
        tmp_week = tmp_datetime.isocalendar().week

        for j in tmp_date:
            dates.append(j)
        
        for k in tmp_time:
            hours.append(k)
        
        for l in tmp_dayofweek:
            dayofweek.append(l)
        
        for m in tmp_week:
            week.append(m)

    # Get rid of duplicate values
    unique_dates = list(set(dates))
    unique_hours = list(set(hours))
    unique_dayofweek = list(set(dayofweek))
    unique_week = list(set(week))

    # Convert to string
    unique_dates = [x.strftime('%m/%d/%Y') for x in unique_dates]
    unique_hours = [x.hour for x in unique_hours]

   # unique_dayofweek = [x for x in unique_dayofweek]


    # Populate our unique_user_daily_freq dict
    for i in unique_dates:
        
        if i not in unique_users_daily_freq:
            unique_users_daily_freq[i] = 1
        else:
            unique_users_daily_freq[i] +=1
        

    # Populate hourly activity
    for j in unique_hours:

        if j not in unique_users_hourly_freq:
            unique_users_hourly_freq[j] = 1
        else:
            unique_users_hourly_freq[j] +=1

    # Populate dayofweeka activity
    for k in unique_dayofweek:

        if k not in unique_users_dayofweek_freq:
            unique_users_dayofweek_freq[k] = 1
        else:
            unique_users_dayofweek_freq[k] +=1
    

    # Populate dayofweeka activity
    for l in unique_week:

        if l not in unique_users_weekly_freq:
            unique_users_weekly_freq[l] = 1
        else:
            unique_users_weekly_freq[l] +=1
    
            
    

In [191]:
# What does the daily user activity look like?
indices = [x for x in unique_users_daily_freq.keys()]
values = [x for x in unique_users_daily_freq.values()]

fig = go.Figure()

# Add traces
fig.add_trace(go.Scatter(x = indices, y = values,
                         mode = 'lines+markers'
                         )

)

fig.show()


In [192]:
# What does the hourly user activity look like?
indices = [x for x in unique_users_hourly_freq.keys()]
values = [x for x in unique_users_hourly_freq.values()]

fig = go.Figure()

# Add traces
fig.add_trace(go.Scatter(x = indices, y = values,
                         mode = 'markers'
                         )

)

fig.show()

In [193]:
# What does the weekly user activity look like?
indices = [x for x in unique_users_weekly_freq.keys()]
values = [x for x in unique_users_weekly_freq.values()]

fig = go.Figure()

# Add traces
fig.add_trace(go.Scatter(x = indices, y = values,
                         mode = 'lines+markers'
                         )

)

fig.show()

In [194]:
# What does the weekly user activity look like?
indices = [x for x in unique_users_dayofweek_freq.keys()]
values = [x for x in unique_users_dayofweek_freq.values()]

fig = go.Figure()

# Add traces
fig.add_trace(go.Scatter(x = indices, y = values,
                         mode = 'markers'
                         )

)

fig.show()

What is the distribution of topics people view?

In [295]:
# Let's get unique_user_ids topics

## Get all unique ids in a list

unique_user_ids = df['user_id'].values[0:1000]

# Create dictionaries
unique_users_topics_freq= {}
unique_topic_scroll_freq = {}
unique_topic_read_freq = {}

for id in unique_user_ids:
    # Get the subset of that user id
    tmp_df = df[df['user_id'] == id]

    # Now lets go through each topic
    indices = np.array(tmp_df.index)
    for i in indices:
        tmp_topics = tmp_df['topics'][i]
        tmp_scroll = tmp_df['scroll_percentage'][i]
        tmp_read = tmp_df['read_time'][i]

        topics = [x for x in tmp_topics]
        scroll = [tmp_scroll]
        read = [tmp_read]

            
    # Find the average scroll percentages across each topic  (Can be related to whether a topic doesnt require too much reading has visualizations)

    ## Group by user ID
    ### Look at article_id for whichever topics the article is included in add that readtime and scroll percentage
        tmp_dict = {k:v for k,v in zip(topics, scroll)}

        for k,v in zip(tmp_dict.keys(), tmp_dict.values()):

            if (k in unique_topic_scroll_freq.keys()):
                tmp_array= np.append(unique_topic_scroll_freq[k],v)
                unique_topic_scroll_freq[k] = tmp_array
            if (k not in unique_topic_scroll_freq.keys()):
                unique_topic_scroll_freq[k] = []

    # Find the average read time across each topic
    ## Group by user ID
    ### Look at article_id for whichever topics the article is included in add that readtime and scroll percentage
        tmp_dict = {k:v for k,v in zip(topics, read)}

        for k,v in zip(tmp_dict.keys(), tmp_dict.values()):

            if (k in unique_topic_read_freq.keys()):
                tmp_array= np.append(unique_topic_read_freq[k],v)
                unique_topic_read_freq[k] = tmp_array
            if (k not in unique_topic_read_freq.keys()):
                unique_topic_read_freq[k] = []

    
    ## Unique User Topics
    # Get rid of duplicate values
    unique_topics = list(set(topics))

    
    # Populate our dict
    for i in unique_topics:
        
        if i not in unique_users_topics_freq:
            unique_users_topics_freq[i] = 1
        else:
            unique_users_topics_freq[i] +=1



In [296]:
# What does the distribution of topics look like lets sort it?
sorted_topic_freq = dict(sorted(unique_users_topics_freq.items(), key = lambda x: x[1], reverse = True))


indices = [x for x in sorted_topic_freq.keys()]
values = [x for x in sorted_topic_freq.values()]

fig = go.Figure()

# Add traces
fig.add_trace(go.Bar(x = indices, y = values,
                         
                         )

)

fig.update_traces(texttemplate='%{text:.2s}', textposition='outside')
fig.update_layout(uniformtext_minsize=8, uniformtext_mode='hide')

fig.show()

In [297]:
# Find the average scroll percentages across each topic

# Get the average scroll length for each article
for k,v in zip(unique_topic_scroll_freq.keys(), unique_topic_scroll_freq.values()):
    unique_topic_scroll_freq[k] = np.nanmean(v)

sorted_unique_topic_scroll_freq = dict(sorted(unique_topic_scroll_freq.items(), key = lambda x: x[1], reverse = True))


indices = [x for x in sorted_unique_topic_scroll_freq . keys()]
values = [x for x in sorted_unique_topic_scroll_freq.values()]

fig = go.Figure()

# Add traces
fig.add_trace(go.Bar(x = indices, y = values,
                         
                         )

)

fig.update_traces(texttemplate='%{text:.2s}', textposition='outside')
fig.update_layout(uniformtext_minsize=8, uniformtext_mode='hide')

fig.show()

In [298]:
# Find the average read times across each topic

# Get the average scroll length for each article
for k,v in zip(unique_topic_read_freq.keys(), unique_topic_read_freq.values()):
    unique_topic_read_freq[k] = np.nanmean(v)

sorted_unique_topic_read_freq = dict(sorted(unique_topic_read_freq.items(), key = lambda x: x[1], reverse = True))


indices = [x for x in sorted_unique_topic_read_freq . keys()]
values = [x for x in sorted_unique_topic_read_freq.values()]

fig = go.Figure()

# Add traces
fig.add_trace(go.Bar(x = indices, y = values,
                         
                         )

)

fig.update_traces(texttemplate='%{text:.2s}', textposition='outside')
fig.update_layout(uniformtext_minsize=8, uniformtext_mode='hide')

fig.show()

In [ ]:
# Average scroll time across each user
df.groupby(by =  'user_id')['scroll_percentage'].mean()

In [243]:
## Average readtime per user
df.groupby(by =  'user_id')['read_time'].mean()

user_id
10068      101.000000
10200       24.000000
10201       87.000000
10623       20.666666
10701       25.500000
              ...    
2589430     66.545456
2589735     86.000000
2590015     62.150002
2590054     21.500000
2590571     16.875000
Name: read_time, Length: 9194, dtype: float32

In [197]:
# For each session, what was the average readtime 

# Distribution blot histogram (bin it)
df.groupby(by= 'session_id')['read_time'].mean()
    

session_id
735         99.0
739        168.0
740         29.0
797         98.0
814         18.0
           ...  
2029646     61.0
2029793    165.0
2029940     19.5
2029941     69.0
2030103    126.0
Name: read_time, Length: 36795, dtype: float32

In [273]:
## For each article, what was each users scroll percentage + read time? 


## Get all unique ids in a list

unique_user_ids = df['user_id'].values[0:1000]

## We take the set because the scroll, article per user is joined in a list for every user id (so just take the set of it!)
unique_user_ids = set(unique_user_ids)


unique_article_ids = df['article_id'].unique()
unique_article_ids= unique_article_ids[~np.isnan(unique_article_ids)]

# Create dictionaries

unique_article_scroll = {k: [0] for k in unique_article_ids}

for id in unique_user_ids:
    # Get the subset of that user id
    tmp_df = df[df['user_id'] == id]

    # Now lets go through each scroll and article
    ## Iterate through each of the df
    indices = np.array(tmp_df.index)
    for i in indices:
        tmp_dict = {}
        # Select the scroll / article of that indice and 
        tmp_scroll = tmp_df['scroll_percentage_fixed'][i]

        tmp_article = tmp_df['article_id_fixed'][i]

        scroll = [x for x in tmp_scroll]
        
        articles = [np.int64(x) for x in tmp_article]
    
        tmp_dict = {k:v for k,v in zip(articles, scroll)}


        for k,v in zip(tmp_dict.keys(), tmp_dict.values()):

            if (k in unique_article_scroll.keys()) & (np.isnan(v) == False):
                tmp_array= np.append(unique_article_scroll[k],v)
                unique_article_scroll[k] = tmp_array
        
    
# Get the average scroll length for each article
for k,v in zip(unique_article_scroll.keys(), unique_article_scroll.values()):
    unique_article_scroll[k] = np.mean(v)



# What does the hourly user activity look like?
indices = [x for x in unique_article_scroll.keys()]
values = [x for x in unique_article_scroll.values()]

fig = go.Figure()

# Add traces
fig.add_trace(go.Scatter(x = indices, y = values,
                         mode = 'markers'
                         )

)

fig.show()


In [240]:
## For each article, what was each users read time


## Get all unique ids in a list

unique_user_ids = df['user_id'].values[0:1000]

## We take the set because the scroll, article per user is joined in a list for every user id (so just take the set of it!)
unique_user_ids = set(unique_user_ids)


unique_article_ids = df['article_id'].unique()
unique_article_ids= unique_article_ids[~np.isnan(unique_article_ids)]


# Create dictionaries

unique_article_read = {k: [0] for k in unique_article_ids}

for id in unique_user_ids:
    # Get the subset of that user id
    tmp_df = df[df['user_id'] == id]

    # Now lets go through each scroll and article
    ## Iterate through each of the df
    indices = np.array(tmp_df.index)
    for i in indices:
        tmp_dict = {}
        # Select the scroll / article of that indice and 
        tmp_read = tmp_df['read_time_fixed'][i]

        tmp_article = tmp_df['article_id_fixed'][i]

        read = [x for x in tmp_read]
        
        articles = [np.int64(x) for x in tmp_article]
    
        tmp_dict = {k:v for k,v in zip(articles, read)}


        for k,v in zip(tmp_dict.keys(), tmp_dict.values()):

            #if unique_article_scroll.get(k) != None:
            #    unique_article_scroll[k] += v
            if (k in unique_article_read.keys()) & (np.isnan(v) == False):
                tmp_array= np.append(unique_article_read[k],v)
                unique_article_read[k] = tmp_array
        
    
    # Get the average scroll length for each article
    for k,v in zip(unique_article_read.keys(), unique_article_read.values()):
        unique_article_read[k] = np.mean(v)



# What does the hourly user activity look like?
indices = [x for x in unique_article_read.keys()]
values = [x for x in unique_article_read.values()]

fig = go.Figure()

# Add traces
fig.add_trace(go.Scatter(x = indices, y = values,
                         mode = 'markers'
                         )

)

fig.show()


In [301]:
# Overall distribtuion of devices

df['device_type'].value_counts()

2    44472
1    23536
3     2413
Name: device_type, dtype: int64

In [ ]:
# Does device type affect topic you're reading? Device type reading? scrolling


In [302]:
# Distribution paid scrubers

In [ ]:
# Do paid subscribers enjoy a specific topic?

In [304]:
# Distribution of Genders
df['gender'].value_counts()

# Gender specific topic 

# Gender readtime / scroll

0.0    4189
1.0     751
Name: gender, dtype: int64

In [ ]:
# Age distribution

# Age topic

# Age readtime/scroll